<a href="https://colab.research.google.com/github/jackodonoghue/A-DS/blob/master/Draughts_2_2_T00194275.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

All Methods work correctly (I think), except for get_adjacent_kills. I added an argument to take in the start position of a piece as I couldn't get it working without.

---


### Rules  

  - Must take a kill 
 
  - If you reach the other side as a normal piece, the move is over and you are kinged 

---



In [ ]:
class CurrentBoard:
  #variables
  NUMBER_OF_COLUMNS=4
  NUMBER_OF_ROWS=8
  might_be_move_here=0

  #constructor
  #setup of board
  def __init__(self,setup_of_board = ["wwww","wwww","wwww","    ","    ","bbbb","bbbb","bbbb"]):
      self.board = setup_of_board

  #helper/general methods
  #display method for the board - return display as 'None' printed after each board when not returning anything
  def display_board(self, game_display = False): 
    def check_char(c):
      if not c.isspace():
        return c  
      else: 
        self.might_be_move_here+=1
        return str(self.might_be_move_here)  
    display="-----------------"
    for row in range(self.NUMBER_OF_ROWS):
      c = self.board[row]
      if row % 2 == 0:
        display+="\n| |{}| |{}| |{}| |{}|".format(c[0], c[1],c[2], c[3])
      else:
        display+="\n|{}| |{}| |{}| |{}| |".format(c[0], c[1],c[2], c[3])
      display+="\n-----------------"
    return display

  def get_position_of_spaces(this):
    spaces=[]
    for row in range(self.NUMBER_OF_ROWS):
      for column in range(self.NUMBER_OF_COLUMNS):
        if self.get_piece_at((row, column)).isspace():
          spaces.append((row, column))
    return spaces

  def get_piece_at(self, position):
    return self.board[position[0]][position[1]]

  def i_for(self, position):
    return position[1]

  def row_for(self, position):
    return position[0]

  def replace_at(self, row_as_string, index, character):
    return row_as_string[:index] + character + row_as_string[index+1:]
  
  def king_me(self, piece): 
   return piece.capitalize()

  def playing_as(self, piece):
    if piece == "W":
      return "w"
    if piece == "B":
      return "b"
    return piece

  def direction_at(self, position):
    #get piece at position
    piece_char = self.get_piece_at(position)

    #if position is upper return 0
    if piece_char.isupper():
      return 0
    #if position is w return 1
    elif piece_char is 'w':
      return 1
    #if position is b return -1
    else: 
      return -1

  def is_opponent_of(self, piece1, piece2):
    return (self.playing_as(piece1) != self.playing_as(piece2)) and piece1 != ' ' and piece2 != ' '

  def state_of_play(self): 
    board_string=(''.join(self.board).lower())

    if 'b' not in board_string:
      return 'W'
    elif 'w' not in board_string:
      return 'B'
    else:
      return 'U'
  
  #moving methods
  #inc = 1 for white ('down the board'), -1 for black ('up' the board), zero for kings (can go up and down)
  #return a list of all possible positions reachable in one move from position  
  #used to create possible moves in search tree
  def all_possible_moves_for_position(self, position, inc):
    all_moves=[]

    #if king do all moves for forwards and back
    if inc == 0:
      return self.all_possible_moves_for_position(position, 1) + self.all_possible_moves_for_position(position, -1)

    #position is a tuple (row, col)
    start_row = position[0]
    start_i = position[1]
    
    new_row = start_row + inc
    #make sure still on board - still within top and bottom of board
    if (new_row >= 0) and (new_row < self.NUMBER_OF_ROWS):
    #can add the current position +- one row 
      all_moves.append((new_row, start_i))
      if (start_row % 2) == 0:
        #if the col is less than position 3 [0,1,2,3] we can add the move in which the piece goes right
        if start_i < self.NUMBER_OF_COLUMNS-1:
          all_moves.append((new_row, start_i+1))
      else:
        #if the col is greater than position 0 [0,1,2,3] we can add the move in which the piece goes 0
        if start_i > 0:
          all_moves.append((new_row, start_i-1))
    return all_moves

  def move_piece_to(self, from_position, to_position):
    board_def = []
    piece = self.get_piece_at(from_position)
    #check if king should happen 
    if (self.row_for(to_position) == self.NUMBER_OF_ROWS-1 and piece == 'w') or (self.row_for(to_position) == 0 and piece == 'b'):
      piece = self.king_me(piece) 
    for row_index in range(self.NUMBER_OF_ROWS):
      row = self.board[row_index]
      if row_index == self.row_for(from_position):
        if row_index == self.row_for(to_position):
          board_def.append(self.replace_at(row, self.i_for(from_position), piece))
        else:          
          board_def.append(self.replace_at(row, self.i_for(from_position), ' '))
      elif row_index == self.row_for(to_position):
        board_def.append(self.replace_at(row, self.i_for(to_position), piece))
      else:
        board_def.append(row)
    return CurrentBoard(board_def)

  #killing methods
  #find the kills for a piece
  def all_kills(self, start_position, current_position, kill_list): #depth first search
    #find kills from current position
    #find nodes
    all_kills_from_here = self.get_adjacent_kills(current_position, kill_list, start_position)
    
    #visited nodes
    all_completed_moves = []

    if all_kills_from_here == []:
      if kill_list == []:
        return []
      else:
        board=self.board_for(start_position, current_position, kill_list)
        return [board]
    
    for kill in all_kills_from_here:            
      more_kills = self.all_kills(start_position, self.jump_to(current_position, kill), kill_list+[kill])
      all_completed_moves.extend(more_kills)

    return all_completed_moves

  #used in all kills
  #added starting position argument. this will allow the piece to jump if the position after the jump is back to the start.
  def get_adjacent_kills(self, position, kill_list, start_position):
    #decide increment - which direction piece can go
    inc = self.direction_at(start_position)
    
    #get kills from position, using increment
    kills=[]
    all_adjacent_squares = self.all_possible_moves_for_position(position, inc)
    
    #check adjacent square
    for square in all_adjacent_squares:
      #check is not in kill list
      if self.is_not_already_killed(square, kill_list):
        if self.is_opponent_of(self.get_piece_at(start_position), self.get_piece_at(square)):
          position_after_jump=self.jump_to(position, square)
          #check jumping wont land on outside of board or on another piece, unless it is the starting position
          if position_after_jump is not None and (self.get_piece_at(position_after_jump) == ' ' or position_after_jump == start_position):
            kills.append(square)
    return kills  

  #jumping over an opisition piece
  def jump_to(self, from_position, over_position):
    #direction we are playing
    row_dir = self.row_for(over_position) - self.row_for(from_position) # will give -1 or +1
    new_row = self.row_for(over_position) + row_dir

    if (new_row < 0) or (new_row > self.NUMBER_OF_ROWS-1):
      return None
    
    #if index's are equal the jump needs to decide left or right from the row 
    if (new_row%2)==0:
      if (self.i_for(from_position) == self.i_for(over_position)):
        new_i = self.i_for(from_position)-1
      else:
        new_i = self.i_for(from_position)+1
    else:
      if (self.i_for(from_position) == self.i_for(over_position)):
        new_i = self.i_for(from_position)+1
      else:
        new_i = self.i_for(from_position)-1
    
    if new_i>=0 and new_i<self.NUMBER_OF_COLUMNS:
      return (new_row, new_i)

  def is_not_already_killed(self, square, list_of_positions): 
    #i.e. position of square not in list of positions 
    return square not in list_of_positions

  #action methods
  # all_possile_moves_for("w") or "b"
  def all_possible_moves_for(self, _this):  
    all_moves = []
    kill_moves = []
    non_kill_moves = []
    
    for row in range(self.NUMBER_OF_ROWS):
      for i in range(self.NUMBER_OF_COLUMNS):
        #checks each position to see if the piece is the same as the piece colour passed in 
        if self.playing_as(self.board[row][i]) == _this:
          #check for kills
          if self.get_adjacent_kills((row, i), [], (row, i)) is not None:
            found_kill=True
          #add kills and non-kills to all moves
          all_moves.append(self.construct_boards_for((row,i) , found_kill))
        found_kill=False
    #add moves to appropriate list
    for board_list in all_moves:
      kill_moves.extend(board_list[0])
      non_kill_moves.extend(board_list[1])
    
    return kill_moves if len(kill_moves) > 0 else non_kill_moves
    #all_moves=[]
    #for kill in kill_moves:
    #  all_moves.append(kill)
    #for move in non_kill_moves:
    #  all_moves.append(move)
    #return { "kill moves": kill_moves, "non-kill moves": non_kill_moves 
    return all_moves

  #construct all boards for all possible moves
  def construct_boards_for(self, from_position, found_kill):
    non_kill_boards=[]
    kill_boards=[]

    #only perform kills or non-kill moves, not both. this is because kills must be taken
    if found_kill:
      kill_boards = self.all_kills(from_position, from_position, [])
    if len(kill_boards) == 0 :
      inc = self.direction_at(from_position)
      for new_position in self.all_possible_moves_for_position(from_position, inc):
        if self.get_piece_at(new_position) == " ":
          non_kill_boards.append(self.move_piece_to(from_position, new_position))
    return kill_boards, non_kill_boards

  #construct board for all kills iteration
  def board_for(self, from_position, end_position, kill_list):
    #construct a board string in which the from  posistion is moved to the end position, removing all pieces in the kill list
    board = self.move_piece_to(from_position, end_position).board

    for kill in kill_list:
      row = board[self.row_for(kill)]
      board[self.row_for(kill)] = self.replace_at(row, self.i_for(kill), " ")
    return CurrentBoard(setup_of_board=board)

In [ ]:
print(CurrentBoard().display_board())

-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| |w|
-----------------
| | | | | | | | |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------


In [ ]:
print(CurrentBoard().display_board())

-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| |w|
-----------------
| | | | | | | | |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------


In [ ]:
class Search_Tree_Node:
  MAX_PLY = 5
  def __init__(self, game , playing_char, ply = 0):
      self.children =[]
      self.ply_depth = ply
      self.value_is_assigned = False
      self.value = 999
      self.board_for_node = game
      self.playing = playing_char
      if self.board_for_node.state_of_play() == "U" and ply <= self.MAX_PLY:
        self.generate_children()
      else:
          if (self.ply_depth % 2) ==0:
            self.value = -1
          else:
            self.value = 1
          self.value_is_assigned = True

  def max_min_value(self):
    # if no value is assigned we will assign the max(or min) of the value of the children
    if  self.value_is_assigned:
        return self.value
    else: 
      self.children = sorted(self.children, key= lambda x: x.max_min_value() )

      if (self.ply_depth % 2) == 0:
        self.value = self.children[-1].value
      else:
        self.value = self.children[0].value
    return self.value

  def other_char(self,current):
    if current == "w":
      return "b"
    if current == "b":
      return "w"
    return " "

  def generate_children(self):
    for next_move in self.board_for_node.all_possible_moves_for(self.playing):
      self.children.append(Search_Tree_Node(next_move, self.other_char(self.playing), self.ply_depth + 1))


In [ ]:
board=CurrentBoard()

In [ ]:
def run_game():
  current_game = CurrentBoard()
  #  Ask player do they want to move first?
  choice = input("Do you want to go first y/n?")
  player_going_first = (choice == "y") or (choice == "Y")

  player_this_turn = player_going_first


  #  Do you want W or B?
  choice = input("Do you want to play W  y/n?")
  player_playing_W = (choice == "y") or (choice == "Y")
  if player_playing_W:
    player_playing = "w"
    computer_playing = "b"
  else:
    player_playing = "b"
    computer_playing = "w"

  #  Loop
  while current_game.state_of_play() == 'U':
   # Display Current board (for player input)  
    print("Current State of Board:\n")
    print(current_game.display_board())
    if player_this_turn:
      print("\nYour possible moves are: \n")
      possible_moves=current_game.all_possible_moves_for(player_playing)
      for move in range(len(possible_moves)):
        print("\n-------", move, "---------\n")
        print(possible_moves[move].display_board())
      player_move = input("Choose your move --")
      index_of_player_move = int(player_move)
      current_game= possible_moves[index_of_player_move]
    else:
      print(" ")
      st = Search_Tree_Node(current_game,computer_playing)
      current_game =  st.children[-1].board_for_node

    player_this_turn = not player_this_turn

    if current_game.state_of_play() !="U":
      if computer_playing.upper() == current_game.state_of_play():
        print(" I win too easy !!!")
      else:
        print("The world is coming to an end!!!")
      current_game.display_board()
      break

In [ ]:
run_game()

Do you want to go first y/n?n
Do you want to play W  y/n?n
Current State of Board:

-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| |w|
-----------------
| | | | | | | | |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
 
Current State of Board:

-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------

Your possible moves are: 


------- 0 ---------

-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| |b| | | | | | |
-----------------
| | |b| |b| |b| |
-----------

In [ ]:
import unittest


board=CurrentBoard()


class TestCurrentBoard(unittest.TestCase):
  
  #variables  
  def test_NUMBER_OF_ROWS(self):
    self.assertEqual(board.NUMBER_OF_ROWS, 8)

  def test_NUMBER_OF_COLUMNS(self):
    self.assertEqual(board.NUMBER_OF_COLUMNS, 4)

  #helper/general methods
  #display_board
  def test_display_board(self):
    self.assertEqual(board.display_board(), "-----------------\n| |w| |w| |w| |w|\n-----------------\n|w| |w| |w| |w| |\n-----------------\n| |w| |w| |w| |w|\n-----------------\n| | | | | | | | |\n-----------------\n| | | | | | | | |\n-----------------\n|b| |b| |b| |b| |\n-----------------\n| |b| |b| |b| |b|\n-----------------\n|b| |b| |b| |b| |\n-----------------")

  #get_piece_at
  def test_get_piece_at_white(self):
    self.assertEqual(board.get_piece_at((0, 0)), 'w')
  
  def test_get_piece_at_black(self):
    self.assertEqual(board.get_piece_at((7,3)), 'b')
  
  def test_get_piece_at_space(self):
    self.assertEqual(board.get_piece_at((3,0)), ' ')
  
  def test_get_piece_at_outside_column_range(self):
    self.assertRaises(IndexError, lambda: board.get_piece_at((8,0)))

  def test_get_piece_at_outside_row_range(self):
    self.assertRaises(IndexError, lambda: board.get_piece_at((0,4)))

  #i_for
  def test_i_for_good(self):
    self.assertEqual(board.i_for((1, 0)), 0)
  
  #row_for
  def test_row_for_good(self):
    self.assertEqual(board.row_for((1, 0)), 1)
  
  #replace_at
  def test_replace_at_good(self):
    self.assertEqual(board.replace_at("wwww", 2, "b"), "wwbw")
    
  def test_king_me_white(self):
    self.assertEqual(board.king_me("w"), "W")

  def test_king_me_black(self):
    self.assertEqual(board.king_me("b"), "B") 

  #playing_as     

  def test_playing_as_small_white(self):
    self.assertEqual(board.playing_as("w"), "w")

  def test_playing_as_king_black(self):
    self.assertEqual(board.playing_as("B"), "b") 
  
  #direction_at

  def test_direction_at_upper(self):
    self.assertEqual(CurrentBoard(setup_of_board = [" W  "," bb ","    "," bb ","    "," bb ","    ","    "]).direction_at((0, 1)), 0)

  def test_direction_at_black(self):
    self.assertEqual(board.direction_at((7,0)), -1)
    
  def test_direction_at_white(self):
    self.assertEqual(board.direction_at((0,0)), 1)

  #is_opponent_of

  def test_is_opponent_of_w_B(self):
    self.assertEqual(board.is_opponent_of("w", "B"), True)
  
  def test_is_opponent_of_W_b(self):
    self.assertEqual(board.is_opponent_of("W", "b"), True)

  def test_is_opponent_of_w_B(self):
    self.assertEqual(board.is_opponent_of("w", "B"), True)
  
  def test_is_opponent_of_w_W(self):
    self.assertEqual(board.is_opponent_of("w", "W"), False)
  
  def test_is_opponent_of_w_space(self):
    self.assertEqual(board.is_opponent_of("w", " "), False)
  
  #state_of_play

  def test_state_of_play_U(self):
    self.assertEqual(board.state_of_play(), "U")

  def test_state_of_play_B(self):
    self.assertEqual(CurrentBoard(setup_of_board=["    "," bb ","    "," bb ","    "," bb ","    ","    "]).state_of_play(), "B")

  def test_state_of_play_W(self):
    self.assertEqual(CurrentBoard(setup_of_board=["  W ","    ","    ","    ","    ","    ","    ","    "]).state_of_play(), "W")

  #moving methods test
  #all_possible_moves_for_position
  def test_all_possible_moves_for_position_white(self):
    self.assertEqual(board.all_possible_moves_for_position((2,0), 1), [(3,0), (3,1)])

  def test_all_possible_moves_for_position_black(self):
    self.assertEqual(board.all_possible_moves_for_position((5,0), -1), [(4,0)])

  def test_all_possible_moves_for_position_edge_of_board(self):
    self.assertEqual(board.all_possible_moves_for_position((0,1), -1), [])
  
  #move_piece_to will move allow 'illegal' moves. It is the responsibility of all_moves/all_kills to ensure legal movement of peices
  def test_move_piece_to(self):
    result=board.move_piece_to((2,0), (3,0))
    self.assertEqual(result.board, ["wwww", "wwww", " www", "w   ", "    ", "bbbb", "bbbb", "bbbb"])

  #all_kills
  #moves will not be made unless kill available
  def test_all_kills_start_game(self):
    self.assertEqual(board.all_kills((3,0),(3,0),[]), [])

  #-----------------
  #| | | |W| | | | |  
  #-----------------
  #| | |b| |b| | | |
  #-----------------
  #| | | | | | | | |
  #-----------------
  #| | |b| |b| | | |
  #-----------------
  #| | | | | | | | |
  #-----------------
  #| | |b| |b| | | |
  #-----------------
  #| | | | | | | | |
  #-----------------
  #| | | | | | | | |
  #-----------------

  def test_all_kills_king_six_black(self):
    kills=CurrentBoard([" W  ", " bb ", "    ", " bb ", "    ", " bb ", "    ", "    "]).all_kills((0,1),(0,1),[])
    self.assertEqual(len(kills), 6)

  def test_all_kills_white_six_black(self):
    kills=CurrentBoard([" w ", " bb ", "    ", " bb ", "    ", " bb ", "    ", "    "]).all_kills((0,1),(0,1),[])
    self.assertEqual(len(kills), 4)

  #-----------------
  #| | | | | | | | |  
  #-----------------
  #| | | | | | | | |
  #-----------------
  #| |w| |w| | | | |
  #-----------------
  #| | | | | | | | |
  #-----------------
  #| |w| |w| | | | |
  #-----------------
  #| | | | | | | | |
  #-----------------
  #| |w| |w| | | | |
  #-----------------
  #| | |b| | | | | |
  #-----------------

  def test_all_kills_black_six_white(self):
    kills=CurrentBoard(["    ", "    ", "ww  ", "    ", "ww  ", "    ", "ww  ", " b  "]).all_kills((7,1),(7,1),[])
    self.assertEqual(len(kills), 4)

  #-----------------                            ----------------- 
  #| |W| | | | | | |                            | | | | | | | |W|  
  #-----------------                            -----------------
  #| | |b| |b| |b| |                            | | | | | | | | |
  #-----------------                            -----------------
  #| | | | | | | | |                            | | | | | | | | |
  #-----------------       move returned
  #| | | | | | | | |     ---------------->      .....
  #-----------------
  #| | | | | | | | |
  #-----------------
  #| | | | | | | | |
  #-----------------
  #| | | | | | | | |
  #-----------------
  #| | | | | | | | |
  #-----------------

  def test_all_kills_king_on_edge_row(self):
    kills=CurrentBoard(["W   ", " bbb", "    ", "    ", "    ", "    ", "    ", "    "]).all_kills((0,0),(0,0),[])
    self.assertEqual(len(kills), 1)

  #-----------------
  #| |w| | | | | | |  
  #-----------------
  #|b| | | | | | | |
  #-----------------
  #| | | | | | | | |
  #-----------------
  #| | | | | | | | |
  #-----------------
  #| | | | | | | | |
  #-----------------
  #| | | | | | | | |
  #-----------------
  #| | | | | | | | |
  #-----------------
  #| | | | | | | | |
  #-----------------

  def test_all_kills_on_edge(self):
    kills=CurrentBoard(["w   ", "b   ", "    ", "    ", "    ", "    ", "    ", "    "]).all_kills((0,0),(0,0),[])
    self.assertEqual(len(kills), 0)


  #all_adjacent_kills - TESTED THOUROUGHLY IN ALL_KILLS
  def test_all_adjacent_kills_with_kill(self):
    kills=CurrentBoard(["w   ", " b  ", "    ", "    ", "    ", "    ", "    ", "    "]).get_adjacent_kills((0,0),[],(0,0))
    self.assertEqual(len(kills), 1)

  def test_all_adjacent_kills_no_kill(self):
    kills=CurrentBoard(["w   ", "b   ", "    ", "    ", "    ", "    ", "    ", "    "]).get_adjacent_kills((0,0),[],(0,0))
    self.assertEqual(len(kills), 0)

  #jump_to - ALREADY TESTED
  def test_all_jump_to_kill(self):
    kills=CurrentBoard(["w   ", " b  ", "    ", "    ", "    ", "    ", "    ", "    "])
    self.assertEqual(kills.jump_to((0,0), (1,1)), (2,1))

  def test_all_jump_to_kill(self):
    kills=CurrentBoard(["w   ", "b   ", "    ", "    ", "    ", "    ", "    ", "    "])
    self.assertEqual(kills.jump_to((0,0), (1,0)), None)
  
  #action methods
  #all_possible_moves_for
  def test_all_possible_moves_for_standard(self):
    self.assertEqual(len(board.all_possible_moves_for('b')["kill moves"]), 0)
    self.assertEqual(len(board.all_possible_moves_for('b')["non-kill moves"]), 7)


  #this board will test multiple moves for white pieces, including king moves, kill moves and ordinary moves
  #-----------------
  #| | | | | | | | |
  #-----------------
  #| | |W| | | | | |
  #-----------------
  #| |b| | | | | | |
  #-----------------
  #| | | | |b| | | |
  #-----------------
  #| | | |W| | | |w|
  #-----------------
  #| | | | | | |b| |
  #-----------------
  #| |w| | | | | | |
  #-----------------
  #| | | | | | | | |
  #----------------- 
  def test_all_possible_moves_for_complex(self):
    complex_board=CurrentBoard(setup_of_board=["    ", " W  ", "b   ", "  b ", " W w ", "   b", "w   ", "    "])
    
    kill_board_strings=[['    ', '    ', '    ', 'W b ', ' W w ', '   b', 'w   ', '    '], ['    ', ' W  ', 'b W ', '    ', '   w ', '   b', 'w   ', '    '], ['    ', ' W  ', 'b   ', '  b ', ' W   ', '    ', 'w w ', '    ']]
    non_kill_board_strings=[['    ', ' W  ', 'b   ', '  b ', ' W w ', '   b', '    ', 'W   '], ['    ', ' W  ', 'b   ', '  b ', ' W w ', '   b', '    ', ' W  ']]
    expected_kills=[]
    expected_moves=[]

    for move in complex_board.all_possible_moves_for('w')["kill moves"]:
      expected_kills.append(move.board)
    for move in complex_board.all_possible_moves_for('w')["non-kill moves"]:
      expected_moves.append(move.board)
    
    self.assertEqual(expected_kills, kill_board_strings)
    self.assertEqual(expected_moves, non_kill_board_strings)
    
  #board_for - tests have been carried out for this method by testing all_possible_moves_for
  #test king wont infinetely circle
  #-----------------
  #| | | |W| | | | |  
  #-----------------
  #| | |b| |b| | | |
  #-----------------
  #| | | | | | | | |
  #-----------------
  #| | |b| |b| | | |
  #-----------------
  #| | | | | | | | |
  #-----------------
  #| | | | | | | | |
  #-----------------
  #| | | | | | | | |
  #-----------------
  #| | | | | | | | |
  #-----------------

  def test_board_for(self):
    kills=CurrentBoard([" W  ", " bb ", "    ", " bb ", "    ", "    ", "    ", "    "]).all_kills((0,1),(0,1),[])
    self.assertEqual(kills[0].board, [" W  ", "    ", "    ", "    ", "    ", "    ", "    ", "    "])

  
  
unittest.main(argv=[''], verbosity=2, exit=False)

In [ ]:
''.join(["a", "e", "i"])

In [ ]:
s=(''.join(["www", "   ", "bbb"]).lower())

if 'b' not in s:
  print('W')
elif 'w' not in s:
  print('B')
else:
  print('U')

In [ ]:
somelist=[1,2,3,4,5,6,7,8,9]
print(somelist)

somelist[:] = [x for x in somelist if not x%2==0]
print(somelist)